In [50]:
import os
import json
import re
import threading
from queue import Queue
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from util.utils import load_file_line_by_line
import logging
from threading import Lock

In [24]:
filterd_abstracts = '/usr/xtmp/hc387/ai_reviewer/data/semantic_scholar/2024_10_8/abstract_filtered_more'

all_abstracts = {}

for dir in os.listdir(filterd_abstracts):
    file_path = os.path.join(filterd_abstracts, dir)
    with open(file_path, 'r') as file:
        data = json.load(file)
    temp_dict = {item['corpusid']: item for item in data}
    all_abstracts.update(temp_dict)

KeyboardInterrupt: 

In [43]:
class combineFiles:
    def __init__(self, categories: dict, main_dict: dict):
        self.categories = categories
        self.main_dict = main_dict
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def process_file(self, file_path, lock):
        self.logger.info(f"Processed file: {file_path}")
        for item in load_file_line_by_line(file_path):
            corpusid = item['corpusid']
            if corpusid in self.main_dict:
                with lock:
                    self.main_dict[corpusid].update(item)
        self.logger.info(f"Finished processing: {file_path}")

    def process_category(self, max_workers):
        """Process all JSON files within a category folder concurrently."""
        lock = Lock()
        for category in self.categories:
            json_files = list(Path(category).glob('*.json'))
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                executor.map(self.process_file, json_files)



{'corpusid': 212987771, 'openaccessinfo': {'externalids': {'MAG': '3006824482', 'ACL': None, 'DOI': None, 'PubMedCentral': None, 'ArXiv': None}, 'license': None, 'url': None, 'status': None}, 'abstract': 'The research presented in this thesis focuses on microbial adaptation to organic chemicals and the resultant effects on ready biodegradation tests. Biodegradability is a key parameter for the environmental risk assessment of organic chemicals. Readily biodegradability is mostly determined using screening tests such as the ready biodegradability tests (RBTs) developed by the Organization for Economic Cooperation and Development (OECD). This thesis confirms that the origin and exposure history of the inocula used in RBTs can lead to highly variable outcomes. Chemostat systems were proposed as a solution to reduce the variability of the test results, by exposing microbial communities to a specific chemicals during a long period of time. First, we tested the impact of prior long-term expo

In [49]:
dict = {
    "abstracts": 1,
    "paper": 2
}

for item in dict:
    print(dict[item])

1
2
